In [ ]:
import sys
import os
import pytest


def pytest_addoption(parser):
    parser.addoption(
        "--target", action="store", default="default_target", help="target directory"
    )
    parser.addoption(
        "--shipping", action="store_true", help="set shader path for shipping"
    )


def pytest_configure(config):
    target = config.getoption("target")
    shipping = config.getoption("shipping")

    target_path = os.path.abspath(
        os.path.join(os.path.dirname(__file__), f"../../../../Binaries/{target}")
    )
    sys.path.append(target_path)
    print(f"Added {target_path} to sys.path")

    package_path = os.path.abspath(
        os.path.join(os.path.dirname(__file__), f"../python")
    )
    sys.path.append(package_path)

    print(f"Added {package_path} to sys.path")

    os.chdir(target_path)

    if shipping:
        shader_path = "./usd/hd_USTC_CG/resources/shaders/"
    else:
        shader_path = "../../source/renderer/nodes/shaders/shaders/"

    config._shader_path = shader_path  # Store shader_path in config object

    print(f"Added {target_path} to sys.path")
    print(f"Shader path set to {shader_path}")


@pytest.fixture
def shader_path(request):
    return request.config._shader_path


In [9]:
import os
import sys
target_path = os.path.abspath(f"../../../../Binaries/Debug")
sys.path.append(target_path)
print(f"Added {target_path} to sys.path")
package_path = os.path.abspath(f"../python")
sys.path.append(package_path)
print(f"Added {package_path} to sys.path")

Added c:\Users\pengfei\WorkSpace\USTC_CG_24\Framework3D\Binaries\Debug to sys.path
Added c:\Users\pengfei\WorkSpace\USTC_CG_24\Framework3D\source\Runtime\renderer\python to sys.path


In [10]:
import glints.scratch_grid
import glints.renderer
import glints.test_utils as test_utils
import torch
import numpy as np
import pytest


In [ ]:


def test_scratch_field():
    field = glints.scratch_grid.ScratchField(10, 5)
    assert field.n == 10
    assert field.m == 5
    assert field.field.shape == (10, 10, 5, 2)


def linear_to_gamma(image):
    return image ** (1.0 / 2.2)


@pytest.mark.skip(reason="Skipping temporarily")
def test_scratch_field_divergence():
    field = glints.scratch_grid.ScratchField(1024, 5)

    optimizer = torch.optim.Adam([field.field], lr=0.01)
    for i in range(1000):

        optimizer.zero_grad()
        divergence, smoothness = field.calc_divergence_smoothness()
        divergence_loss = torch.mean(divergence**2)
        divergence_loss.backward()
        optimizer.step()

        print("divergence_loss", divergence_loss.item())

    test_utils.save_image(divergence[:, :, 0], [1024, 1024], "divergence.exr")

    r = glints.renderer.Renderer()
    vertices, indices = glints.renderer.plane_board_scene_vertices_and_indices()
    camera_position_np = np.array([4.0, 0.1, 2.5], dtype=np.float32)
    r.set_camera_position(camera_position_np)
    fov_in_degrees = 35
    resolution = [1536, 1024]
    r.set_perspective(
        np.pi * fov_in_degrees / 180.0, resolution[0] / resolution[1], 0.1, 1000.0
    )
    r.set_mesh(vertices, indices)
    r.set_light_position(torch.tensor([4.0, -0.1, 2.5], device="cuda"))

    r.set_width(torch.tensor([0.001], device="cuda"))

    image = glints.scratch_grid.render_scratch_field(r, resolution, field)

    test_utils.save_image(image, resolution, "test_divergence_scratch_field.exr")


def test_render_scratch_field():
    r = glints.renderer.Renderer()

    vertices, indices = glints.renderer.plane_board_scene_vertices_and_indices()
    camera_position_np = np.array([4.0, 0.1, 2.5], dtype=np.float32)
    r.set_camera_position(camera_position_np)
    fov_in_degrees = 35
    resolution = [768, 512]
    r.set_perspective(
        np.pi * fov_in_degrees / 180.0, resolution[0] / resolution[1], 0.1, 1000.0
    )
    r.set_mesh(vertices, indices)
    r.set_light_position(torch.tensor([4.0, -0.1, 2.5], device="cuda"))

    r.set_width(torch.tensor([0.001], device="cuda"))

    field = glints.scratch_grid.ScratchField(256, 2)
    image = glints.scratch_grid.render_scratch_field(r, resolution, field)
    test_utils.save_image(image, resolution, "scratch_field_initial.exr")
    target_image = r.prepare_target("texture.png", resolution)
    loss_fn = torch.nn.L1Loss()
    regularization_loss_fn = torch.nn.L1Loss()
    regularizer = torch.optim.Adam([field.field], lr=0.005)

    for i in range(400):

        regularizer.zero_grad()
        divergence, smoothness = field.calc_divergence_smoothness()

        loss_divergence = regularization_loss_fn(
            divergence, torch.zeros_like(divergence)
        )

        loss_smoothness = regularization_loss_fn(
            smoothness, torch.zeros_like(smoothness)
        )

        resularization_loss = loss_divergence + loss_smoothness
        if i == 0:
            old_regularization_loss = resularization_loss.item()
        resularization_loss.backward()

        regularizer.step()
        field.fix_direction()
    optimizer = torch.optim.Adam([field.field], lr=0.04)
    torch.set_printoptions(precision=5)
    for _ in range(150):  # Number of optimization steps

        optimizer.zero_grad()
        image = glints.scratch_grid.render_scratch_field(r, resolution, field)
        loss_image = loss_fn(linear_to_gamma(image), target_image) * 1000
        density_loss = torch.mean(
            torch.norm(field.field, dim=3) * 0.1
        )  # less scratches, better direction

        total_loss = loss_image + density_loss
        total_loss.backward()
        optimizer.step()

        resularization_loss = torch.tensor(10000000000000.0)

        regularization_steps = 0

        while resularization_loss.item() > old_regularization_loss * 0.1:
            # for i in range(30):

            regularizer.zero_grad()
            divergence, smoothness = field.calc_divergence_smoothness()
            loss_divergence = regularization_loss_fn(
                divergence, torch.zeros_like(divergence)
            )

            loss_smoothness = regularization_loss_fn(
                smoothness, torch.zeros_like(smoothness)
            )

            resularization_loss = loss_divergence + loss_smoothness
            resularization_loss.backward()

            regularizer.step()
            regularization_steps += 1


        print(
            "iteration:",
            _,
            "loss_divergence",
            loss_divergence.item(),
            "loss_smoothness",
            loss_smoothness.item(),
            "density_loss",
            density_loss.item(),
            "loss_image",
            loss_image.item(),
            "total_loss",
            total_loss,
            "regularization_steps",
            regularization_steps
        )

    field.fix_direction()

    for i in range(2):
        test_utils.save_image(
            1000 * divergence[:, :, i], resolution, f"divergence_{i}.exr"
        )
        test_utils.save_image(
            100 * smoothness[:, :, i], resolution, f"smoothness_{i}.exr"
        )

        density = torch.norm(field.field[:, :, i], dim=2)

        directions = field.field[:, :, i] / density.unsqueeze(2)  # shape [n,n,2]
        # expand 1 dimension to 3 dimensions
        directions = torch.cat(
            [directions, torch.zeros_like(directions[:, :, :1])], dim=2
        )
        test_utils.save_image(directions, resolution, f"directions_{i}.exr")

        test_utils.save_image(density, resolution, f"density_{i}.exr")
        test_utils.save_image(field.field[:, :, i, :1], resolution, f"field_{i}.exr")

    image = glints.scratch_grid.render_scratch_field(r, resolution, field)

    test_utils.save_image(image, resolution, "scratch_field.exr")
